# SIIM-ISIC Melanoma Classification SoSe24

Der Folgende Block lädt relevante Libraries, die für dieses Projekt benötigt werden. Besondere sind: 

**ThreadPoolExecutor**

- Multithreaden von rechenschweren Prozessen
- Beschleunigen von Ladeaufgaben


**tqdm** 

- Anzeigen von Ladebalken. 
- Weiteres Feedback, wenn ein Prozess beendet wird


----
<span style="color:grey">


**Kommentar:**

Es können Artefakte in Form von importierten Libraries geben, die nicht weiter ausgeführt werden.
</span>

In [ ]:
import os
import datetime
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore
from tensorflow.keras import models, layers, optimizers  # type: ignore
from tensorflow.keras.models import load_model  # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore

warnings.filterwarnings('ignore')

datetime.datetime.now().strftime("Fertiggestellt um %X den %x")

# Laden der Bilder und dazugehörigen CSV Dateien. 
Der folgende Block lädt Bilder und CSV Daten. Diese werden multithreaded, um die Ladezeiten um etwa das x-Fache der verfügbaren Prozessorkerne zu verkürzen.

Bilder werden in 128x128 mit Farbe in den RAM geladen, zusätzlich werden die **Namen** und **target** Labels aus der csv-Datei extrahiert und mit den Bildern synchronisiert.

Im letzten Schritt werden die Trainingsbilder in Trainings- und Validierungssätzen aufgeteilt. Hierbei werden 20% der Trainingsdaten zu Validierungsdaten allokiert.

In [ ]:
path_data_train = '/kaggle/input/siim-isic-melanoma-classification/train.csv'
path_image_train = '/kaggle/input/siim-isic-melanoma-classification/jpeg/train'

# Bilder und .csv Dateien laden
def load_image(image_path, target_size=(128, 128)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    return np.array(image)

def load_data(csv_file, image_dir, max_images=None, target_size=(128, 128), num_workers=None):
    data = pd.read_csv(csv_file)
    if max_images is not None:
        data = data.head(max_images)

    # Bildname und target extrahieren
    image_names = data['image_name'].values
    targets = data['target'].values

    images = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = []
        for image_name in image_names:
            image_path = os.path.join(image_dir, image_name + '.jpg')
            futures.append(executor.submit(load_image, image_path, target_size))

        for future in tqdm(futures, desc="Lade Bilder", total=len(futures)):
            images.append(future.result())

    return np.array(images), np.array(targets), image_names

num_workers = os.cpu_count() or 1   # Setze Threads auf alle erkannten Kerne, ansonsten 1

# Laden der Trainingsdaten: Bildname, Bild, target
train_images, train_targets, train_image_names = load_data(path_data_train, path_image_train, max_images=None, num_workers=num_workers)

# Aufteilen der Trainingsdaten in Trainings- und Validierungssätze
train_images, val_images, train_targets, val_targets, train_image_names, val_image_names = train_test_split(train_images, 
                                                                                                            train_targets, 
                                                                                                            train_image_names, 
                                                                                                            test_size=0.2,
                                                                                                            random_state=42, 
                                                                                                            stratify=train_targets)

# Verteilung der Labels anzeigen
print("Verteilung der Trainingslabels:", np.bincount(train_targets))
print("Verteilung der Validierungslabels:", np.bincount(val_targets))

datetime.datetime.now().strftime("Fertiggestellt um %X den %x")

# Datenaugmentierung von Trainings- und Validierungsdaten
Der folgende Block verändert die Verteilung der Trainings- und Validierungsdaten von einer Ungleichheit von '98% zu 2%' zu einer ausgewogeneren Verteilung von '50% zu 50%'. Dies wird durchgeführt, um zu verhindern, dass das Modell dazu neigt, überrepräsentierte Klassen, wie Nullen, zu bevorzugen.



In [ ]:
# Parameter zur Datenaugemntierung festlegen
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

def augment_class_images(class_images, augment_size, generator):
    augmented_images = []
    for i in tqdm(range(augment_size), desc="Augmentiere Bilder"):
        augmented_image = generator.random_transform(class_images[i % len(class_images)])
        augmented_images.append(augmented_image)
    return np.array(augmented_images)

# Bösartige Bilder ermitteln
train_minority_class_images = train_images[train_targets == 1]
val_minority_class_images = val_images[val_targets == 1]

# Augmentationsgröße auf eine Verteilung von 50-50 einstellen
train_augment_size = len(train_images[train_targets == 0]) - len(train_minority_class_images)
val_augment_size = len(val_images[val_targets == 0]) - len(val_minority_class_images)

# Trainings- und Validierungsdaten augmentieren
train_augmented_images = augment_class_images(train_minority_class_images, train_augment_size, datagen)
train_augmented_targets = np.ones(train_augment_size)
val_augmented_images = augment_class_images(val_minority_class_images, val_augment_size, datagen)
val_augmented_targets = np.ones(val_augment_size)

# Kombinieren der augmentierten Bilder mit den ursprünglichen Daten
train_images_balanced = np.concatenate((train_images, train_augmented_images), axis=0)
train_targets_balanced = np.concatenate((train_targets, train_augmented_targets), axis=0)
val_images_balanced = np.concatenate((val_images, val_augmented_images), axis=0)
val_targets_balanced = np.concatenate((val_targets, val_augmented_targets), axis=0)

# Mischen der Daten
def shuffle_data(images, targets):
    indices = np.arange(images.shape[0])
    np.random.shuffle(indices)
    return images[indices], targets[indices]

train_images_balanced, train_targets_balanced = shuffle_data(train_images_balanced, train_targets_balanced)
val_images_balanced, val_targets_balanced = shuffle_data(val_images_balanced, val_targets_balanced)

train_targets_balanced = train_targets_balanced.astype(int)
val_targets_balanced = val_targets_balanced.astype(int)

# Verteilung der Labels nach der Augmentation anzeigen
print("Verteilung der Trainingslabels nach der Augmentation:", np.bincount(train_targets_balanced))
print("Verteilung der Validierungslabels nach der Augmentation:", np.bincount(val_targets_balanced))

datetime.datetime.now().strftime("Fertiggestellt um %X den %x")

# Erstellen eines CNN Models
Der folgende Block definiert ein CNN (Convolutional Neural Network). Mit einem Input von 128x128 Pixeln und 3 Farbkanälen und setzt sich aus drei Convolutional und drei MaxPooling Schichten zusammen. 

Die Conv2D Schichten starten bei 32 und verdoppeln sich pro Ebene bis 256. 

MaxPooling wird zwischen die Conv Schichten geschaltet, um die räumliche Größe auf die wichtigsten Pixel zu halbieren.

Flatten transformiert die 3 Dimensionale Matrix in eine 1 Dimensionale, um diese an die folgende Dense Schicht verbinden zu können.

Dropout deaktiviert 30% zufälliger Neuronen, um Overfitting zu vermeiden.

Die letzte Schicht beschreibt die Ausgabe Schicht und gibt nur binäre Werte wieder.


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])
model.summary()

# Kompilieren und trainieren des Models

In [ ]:
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Callback
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Model trainieren
history = model.fit(train_images_balanced, 
                    train_targets_balanced, 
                    batch_size=32,
                    epochs=100, 
                    validation_data=(val_images_balanced, val_targets_balanced))

datetime.datetime.now().strftime("Fertiggestellt um %X den %x")


# Model speichern

In [ ]:
model.save("/kaggle/working/modelfunktioniert.h5")
datetime.datetime.now().strftime("Fertiggestellt um %X den %x")

In [ ]:
import gc

del train_images
# del train_targets
del val_images
# del val_targets

gc.collect()

# Visualisierung der Ergebnisse

In [ ]:
# Visualisierung der Trainings- und Validierungsverluste
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, max(history.history['loss'] + history.history['val_loss']))
plt.legend()
plt.show()

# Visualisierung der Trainings- und Validierungsgenauigkeit
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim(0, 1)
plt.legend()
plt.show()

## Die Konfusionsmatrix zeigt hohe Werte in TP und TN an, was bedeutet, dass das Modell richtig funktioniert

In [ ]:
# Vorhersagen auf den augmentierten Validierungsdaten
val_predictions = (model.predict(val_images_balanced) > 0.5).astype("int32")

# Berechnung der Genauigkeit
accuracy = accuracy_score(val_targets_balanced, val_predictions)
print(f"Genauigkeit: {accuracy:.2f}")

# Confusion Matrix berechnen
conf_matrix = confusion_matrix(val_targets_balanced, val_predictions)

# Berechnung von TP, TN, FP und FN
TP = conf_matrix[1, 1]
TN = conf_matrix[0, 0]
FP = conf_matrix[0, 1]
FN = conf_matrix[1, 0]

# Plot der Confusion Matrix mit Werten
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap='Blues')
plt.title('Konfusionsmatrix')
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ['Negativ', 'Positiv'])
plt.yticks(tick_marks, ['Negativ', 'Positiv'])
plt.xlabel('Wahre Werte')
plt.ylabel('Vorhersage')

# Anzeigen von TP, TN, FP und FN
plt.text(0, 0, f"True Negatives (TN): {TN}", horizontalalignment='center', verticalalignment='center', color='black')
plt.text(1, 1, f"True Positives (TP): {TP}", horizontalalignment='center', verticalalignment='center', color='black')
plt.text(0, 1, f"False Positives (FP): {FP}", horizontalalignment='center', verticalalignment='center', color='black')
plt.text(1, 0, f"False Negatives (FN): {FN}", horizontalalignment='center', verticalalignment='center', color='black')

plt.show()


# Vorbereitung für die Submission in Kaggle
Die Testbilder werden geladen, die Namen aus den Bildpfaden extrahiert und in einer Datei submission.csv unter image_name mit den entsprechenden Targets des Modells gespeichert.

| image_name | target |
| ----------- | ----------- |
|ISIC_XXXXXXX|0|
|ISIC_XXXXXXX|0|
|ISIC_XXXXXXX|1|
|ISIC_XXXXXXX|0|
|ISIC_XXXXXXX|0|

In [ ]:
path_image_test = '/kaggle/input/siim-isic-melanoma-classification/jpeg/test'
# path_model = '/kaggle/input/model1/tensorflow1/model_abgabe_97_prozent/1/modelfunktioniert.h5' # Vorerstelltes Modell
path_model = '/kaggle/working/modelfunktioniert.h5' # Neuerstelltes Modell, für submission

# Funktion zum Laden von Bildern
def load_image(image_path, target_size=(128, 128)):
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    return np.array(image)

# Modell laden
model = load_model(path_model)

# Testbilder laden
def load_test_images(image_dir, target_size=(128, 128), num_workers=None):
    image_names = os.listdir(image_dir)
    images = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = []
        for image_name in image_names:
            image_path = os.path.join(image_dir, image_name)
            futures.append(executor.submit(load_image, image_path, target_size))

        for future in tqdm(futures, desc="Lade Testbilder", total=len(futures)):
            images.append(future.result())

    return np.array(images), image_names

# Anzahl der Prozesse oder Threads einstellen
num_workers = os.cpu_count() or 1   # Alle verfügbaren Kerne, die das System erkennt, ansonsten 1

# Laden der Testbilder
test_images, test_image_names = load_test_images(path_image_test, num_workers=num_workers)

datetime.datetime.now().strftime("Fertiggestellt um %X den %x")

In [ ]:
# Vorhersagen auf den Testbildern
test_predictions = (model.predict(test_images) > 0.5).astype("int32")

# Entfernen der ".jpg"-Erweiterung von den Bildnamen
test_image_names = [os.path.splitext(name)[0] for name in test_image_names]

# Ergebnisse in eine DataFrame speichern
results = pd.DataFrame({'image_name': test_image_names, 'target': test_predictions.flatten()})

# Speichern der Ergebnisse in eine CSV-Datei
output_path = '/kaggle/working/submission.csv'
results.to_csv(output_path, index=False)

datetime.datetime.now().strftime("Fertiggestellt um %X den %x")

## Überprüfen, wie die Testdaten verteilt sind und ob diese Korrekt gespeichert wurden

In [ ]:
submission_df = pd.read_csv(output_path)

print(submission_df.head())

counts = submission_df['target'].value_counts()

# Ausgabe der Ergebnisse
print(f"Anzahl der 0 in den Vorhersagen: {counts.get(0, 0)}")
print(f"Anzahl der 1 in den Vorhersagen: {counts.get(1, 1)}")